In [24]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import pickle
import os

import sys
sys.path.append(os.path.join('..', '..'))

from global_params import *

In [25]:
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [26]:
target_size = (28, 28)
batch_size = 200

In [27]:
train_path = os.path.join('..', '..', 'data', 'train')

train_batches = keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1
).flow_from_directory(
    train_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='sparse'
)

Found 5783 images belonging to 901 classes.


In [28]:
# Callback for early stopping
es_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=5,
    restore_best_weights=True
)

In [34]:
# Hyperparameters
epochs = 100
learning_rate = 1e-2
validation_split = 0.1
# Math works out so that one epoch equals one pass through the training data.
train_steps_per_epoch = train_batches.n // batch_size

In [30]:
model = keras.Sequential([
    keras.layers.Conv2D(64, (7, 7), activation='relu', strides=(2, 2), padding='same', input_shape=(target_size[0], target_size[1], 3), name='conv1'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool1'),
    keras.layers.BatchNormalization(name='norm1'),
    
    keras.layers.Conv2D(64, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv2a'),
    keras.layers.Conv2D(192, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv2'),
    keras.layers.BatchNormalization(name='norm2'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool2'),
    
    keras.layers.Conv2D(192, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv3a'),
    keras.layers.Conv2D(384, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv3'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool3'),
    
    keras.layers.Conv2D(384, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv4a'),
    keras.layers.Conv2D(256, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv4'),
    
    keras.layers.Conv2D(256, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv5a'),
    keras.layers.Conv2D(256, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv5'),
    
    keras.layers.Conv2D(256, (1, 1), activation='relu', strides=(1, 1), padding='same', name='conv6a'),
    keras.layers.Conv2D(256, (3, 3), activation='relu', strides=(1, 1), padding='same', name='conv6'),
    keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', name='pool4'),

    keras.layers.Flatten(name='flatten'),
    
    keras.layers.Dense(32 * 128, activation='relu', name='fc1'),
    keras.layers.Dense(32 * 128, activation='relu', name='fc2'),
    keras.layers.Dense(128, activation='relu', name='fc3'),
    
    keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1), name='l2')
], name='zeiler_fergus')

In [31]:
model.summary()

Model: "zeiler_fergus"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 14, 14, 64)        9472      
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 7, 7, 64)          0         
_________________________________________________________________
norm1 (BatchNormalization)   (None, 7, 7, 64)          256       
_________________________________________________________________
conv2a (Conv2D)              (None, 7, 7, 64)          4160      
_________________________________________________________________
conv2 (Conv2D)               (None, 7, 7, 192)         110784    
_________________________________________________________________
norm2 (BatchNormalization)   (None, 7, 7, 192)         768       
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 4, 4, 192)       

In [36]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=learning_rate),
    loss=tfa.losses.TripletSemiHardLoss()
)

In [37]:
# %%script echo "Comment line with %%script echo to run this cell."

history = model.fit(
    train_batches,
    epochs=epochs,
    steps_per_epoch=train_steps_per_epoch,
    # callbacks=[es_callback]
)

Epoch 1/100
28/28 [==============================] - 75s 3s/step - loss: 0.9994
Epoch 2/100
28/28 [==============================] - 69s 2s/step - loss: 0.9992
Epoch 3/100
28/28 [==============================] - 67s 2s/step - loss: 0.9982
Epoch 4/100
28/28 [==============================] - 68s 2s/step - loss: 0.9986
Epoch 5/100
28/28 [==============================] - 67s 2s/step - loss: 0.9996
Epoch 6/100
28/28 [==============================] - 71s 3s/step - loss: 0.9982
Epoch 7/100
28/28 [==============================] - 69s 2s/step - loss: 0.9998
Epoch 8/100
28/28 [==============================] - 75s 3s/step - loss: 0.9732
Epoch 9/100
 3/28 [==>...........................] - ETA: 53s - loss: 0.9999

KeyboardInterrupt: 

In [ ]:
%%script echo "Comment line with %%script echo to run this cell."

model.save('model.h5')

In [ ]:
%%script echo "Comment line with %%script echo to run this cell."

with open('history.pickle', 'wb') as f:
    pickle.dump(history.history, f)

In [ ]:
model = keras.models.load_model('model.h5')

In [ ]:
with open('history.pickle', 'rb') as f:
    history = pickle.load(f)